In [ ]:
import os

import pycwb
from pycwb.config import Config
from pycwb.modules.logger import logger_init

logger_init()

config = Config('./user_parameters_injection.yaml')
config.ifo, config.injection

## generate injected data for each detector with given parameters in config

In [ ]:
from pycwb.modules.read_data import generate_injection, generate_noise_for_job_seg
from pycwb.modules.job_segment import create_job_segment_from_config

job_segments = create_job_segment_from_config(config)

data = generate_noise_for_job_seg(job_segments[0], config.inRate)
data = generate_injection(config, job_segments[0], data)
data

In [ ]:
%matplotlib inline
# plot the pycbc time series for L1
data[0].plot()

## apply data conditioning to the data

In [ ]:
from pycwb.modules.data_conditioning import data_conditioning
from pycwb.modules.plot import plot_spectrogram

strains, nRMS = data_conditioning(config, data)

# plot the spectrogram for L1
plt = plot_spectrogram(strains[0], gwpy_plot=True)
ax = plt.gca()
ax.set_ylim(15,1024)

strains, nRMS

## calculate coherence

In [ ]:
from pycwb.modules.coherence import coherence

# calculate coherence
fragment_clusters = coherence(config, strains, nRMS)


## supercluster

In [ ]:
from pycwb.modules.super_cluster import supercluster
from pycwb.types.network import Network

network = Network(config, strains, nRMS)

pwc_list = supercluster(config, network, fragment_clusters, strains)

## likelihood

In [ ]:
from pycwb.modules.likelihood import likelihood

events, clusters, skymap_statistics = likelihood(config, network, pwc_list)

In [ ]:
%matplotlib inline
from pycwb.modules.plot import plot_event_on_spectrogram

for i, tf_map in enumerate(strains):
    plt = plot_event_on_spectrogram(tf_map, events)
    plt.show()

## plot statistics

In [ ]:
%matplotlib inline
from gwpy.spectrogram import Spectrogram

for cluster in clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("likelihood")

    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()

In [ ]:
%matplotlib inline
from gwpy.spectrogram import Spectrogram

for cluster in clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("null")

    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()